# CS 20 : TensorFlow for Deep Learning Research
## Lecture 04 : Eager execution
### Custon training walkthrough
Categorizing Iris flowers by species by using Tensorflow's eager execution.

This guide uses these high-level TensorFlow concepts:

* Enable an [eager execution](https://www.tensorflow.org/guide/eager?hl=ko) development environment,
* Import data with the [Datasets API](https://www.tensorflow.org/guide/datasets?hl=ko)
* Build models and layers with TensorFlow's [Keras API](https://keras.io/getting-started/sequential-model-guide/)  
  
  
* Reference
    + https://www.tensorflow.org/tutorials/eager/custom_training_walkthrough?hl=ko

### Setup

In [1]:
from __future__ import absolute_import, division, print_function
import os, sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

tf.enable_eager_execution()

### Create a model

In [2]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu, input_shape = (4,)))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3))

In [3]:
def loss_fn(model, features, label):
    score = model(features)
    return tf.losses.sparse_softmax_cross_entropy(labels = label, logits = score)

In [4]:
print(model.trainable_variables)

[<tf.Variable 'dense/kernel:0' shape=(4, 10) dtype=float32, numpy=
array([[ 0.06804562, -0.45806283, -0.11035424, -0.64274293,  0.18820602,
         0.0355705 ,  0.23375762,  0.6061579 , -0.36892197, -0.5656868 ],
       [-0.09489417, -0.28409988, -0.44565356,  0.11373091, -0.12645435,
        -0.51101196,  0.5629221 , -0.38564408,  0.25186807,  0.19694126],
       [ 0.12437344,  0.4322877 , -0.52797174, -0.2261037 , -0.62911326,
        -0.07957786, -0.03850067, -0.24269658, -0.04691249, -0.10672408],
       [ 0.6275412 ,  0.36151087, -0.57334185, -0.45194614, -0.43017197,
         0.12567282, -0.2393085 ,  0.04838419,  0.52742684,  0.5202192 ]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
array([[-0.44789726, -0.15868947, -0.11684254,  0.40309513, -0.12921724,
         0.02773911,  0.07647151, -0.5429949 ,  0.397

### Import and parse the training dataset

In [5]:
os.listdir('../data/lecture04/')

['iris_training.csv', 'iris_test.csv']

In [6]:
# define parsing function
def parse_single_example(record):
    decoded = tf.decode_csv(record, [[.0],[.0],[.0],[.0],[]])
    features = decoded[:4]
    label = tf.cast(x = decoded[4], dtype = tf.int32)
    return features, label

In [7]:
epochs = 10
batch_size = 8
learning_rate = .03
tr_dataset = tf.data.TextLineDataset(filenames = '../data/lecture04/iris_training.csv')
tr_dataset = tr_dataset.map(parse_single_example)
tr_dataset = tr_dataset.shuffle(200).batch(batch_size = batch_size)
opt = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
global_step = tf.Variable(0.)

### Train the model

In [8]:
for epoch in range(epochs):
    avg_loss = 0
    tr_step = 0
    
    for mb_x, mb_y in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, mb_x, mb_y)
        grads = tape.gradient(tr_loss, model.variables)
        opt.apply_gradients(zip(grads, model.variables), global_step = global_step)
        
        avg_loss += tr_loss
        tr_step += 1
    else:
        avg_loss /= tr_step
    
    print('epoch : {:3}, ce_loss : {:.3f}'.format(epoch + 1, avg_loss))

epoch :   1, ce_loss : 0.970
epoch :   2, ce_loss : 0.805
epoch :   3, ce_loss : 0.725
epoch :   4, ce_loss : 0.657
epoch :   5, ce_loss : 0.570
epoch :   6, ce_loss : 0.494
epoch :   7, ce_loss : 0.435
epoch :   8, ce_loss : 0.391
epoch :   9, ce_loss : 0.401
epoch :  10, ce_loss : 0.388


### Evaluate the model on the test dataset

In [9]:
tst_dataset = tf.data.TextLineDataset(filenames = '../data/lecture04/iris_test.csv')
tst_dataset = tst_dataset.map(parse_single_example)
tst_dataset = tst_dataset.batch(batch_size = 30)

In [10]:
tst_x, tst_y = next(iter(tst_dataset))
tst_yhat = tf.argmax(model(tst_x), axis = -1, output_type = tf.int32) 

In [11]:
print('test accuracy : {:.2%}'.format(np.mean(tf.equal(tst_y, tst_yhat))))

test accuracy : 96.67%
